# Incremental Training on Large Datasets

Some scikit-learn estimators implement a `partial_fit` method.
This means they can be trained incrementally. The `Incremental` meta-estimator in
Dask-ML provides a nice bridge between data stored in a Dask Array and estimators implementing `partial_fit`.

In [1]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://dask-scheduler:8786 Dashboard: http://dask-scheduler:8787/status,Cluster Workers: 10 Cores: 40 Memory: 128.00 GB


In [2]:
import dask
import dask.array as da
from distributed.utils import format_bytes

import dask_ml.datasets
import dask_ml.model_selection

We'll generate a large random dataset. In practice, you would load data from a shared file system.

In [3]:
n_samples = 4_000_000
n_features = 1_000
chunks = n_samples // 50

X, y = dask_ml.datasets.make_classification(
    n_samples=n_samples, n_features=n_features,
    chunks=chunks,
    random_state=0
)

We'll split the data into train and test sets.

In [4]:
X_train, X_test, y_train, y_test = dask_ml.model_selection.train_test_split(
    X, y
)
X_train, X_test, y_train, y_test = dask.persist(
    X_train, X_test, y_train, y_test
)

In [5]:
format_bytes(X_train.nbytes)

'28.80 GB'

We'll use an `SGDClassifier` for the underlying estimator implementing `partial_fit`.
`Incemental` will sequentially pass blocks of the dask array to it.

In [6]:
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental

In [7]:
estimator = SGDClassifier(
    max_iter=1000,
    random_state=0
)

inc = Incremental(estimator, scoring='accuracy')

In [11]:
%time _ = inc.fit(X_train, y_train, classes=[0, 1])

CPU times: user 49 ms, sys: 6 ms, total: 55 ms
Wall time: 16.6 s


Prediction is lazy and returns a Dask Array. It's generally preferable to keep data on the cluster, rather than trying to bring large results back to a single machine.

In [14]:
inc.predict(X_test)

dask.array<predict, shape=(400000,), dtype=int64, chunksize=(8000,)>

Scoring is immediate, but happens on the cluster.

In [16]:
%time inc.score(X_test, y_test)

CPU times: user 48 ms, sys: 6 ms, total: 54 ms
Wall time: 352 ms


0.51213

So this model isn't particularly good, but we were able to score a large dataset quickly.